In [1]:
import logging
from pathlib import Path

from farm.data_handler.data_silo import DataSilo
from farm.data_handler.processor import TextClassificationProcessor
from farm.modeling.optimization import initialize_optimizer
from farm.infer import Inferencer
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.tokenization import Tokenizer
from farm.train import Trainer
from farm.utils import set_all_seeds, MLFlowLogger, initialize_device_settings

04/22/2021 23:26:34 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO)

ml_logger = MLFlowLogger(tracking_uri="./mlruns/")
ml_logger.init_experiment(experiment_name="Fake_News", run_name="on_base")



 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 


In [3]:
set_all_seeds(seed=42)
n_epochs = 1
batch_size = 16
evaluate_every = 20
lang_model = "bert-base-uncased"
do_lower_case = True
# or a local path:
# lang_model = Path("./saved_models/bert-english-lm-tutorial")
use_amp = None

device, n_gpu = initialize_device_settings(use_cuda=True, use_amp=use_amp)

04/22/2021 23:26:37 - INFO - farm.utils -   Using device: CUDA 
04/22/2021 23:26:37 - INFO - farm.utils -   Number of GPUs: 1
04/22/2021 23:26:37 - INFO - farm.utils -   Distributed Training: False
04/22/2021 23:26:37 - INFO - farm.utils -   Automatic Mixed Precision: None


In [4]:
# 1.Create a tokenizer
tokenizer = Tokenizer.load(pretrained_model_name_or_path=lang_model, do_lower_case=do_lower_case)

04/22/2021 23:26:51 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'


In [5]:
# 2. Create a DataProcessor that handles all the conversion from raw text into a pytorch Dataset
# Here we load GermEval 2018 Data automaticaly if it is not available.
# GermEval 2018 only has train.tsv and test.tsv dataset - no dev.tsv

# https://projects.fzai.h-da.de/iggsa/germeval-2018/

label_list = ["REAL", "OTHER"]
metric = "acc"
# metric = "f1_macro"
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir=Path("./data/fakenews"),
                                        label_list=label_list,
                                        metric=metric,
                                        label_column_name="is_label",
                                        multilabel=False
                                        #text_column_name="text"
                                        )

In [6]:
# 3. Create a DataSilo that loads several datasets (train/dev/test), provides DataLoaders for them and calculates a
#    few descriptive statistics of our datasets
data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)


04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -   LOADING TRAIN DATA
04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -   ==================
04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -   Loading train set from: S:\Hub\mittelstandai\hoftest\data\fakenews\train.tsv 
04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -   Got ya 3 parallel workers to convert 35918 dictionaries to pytorch datasets (chunksize = 2000)...
04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -    0    0    0 
04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -   /w\  /w\  /w\
04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -   /'\  / \  /'\
04/22/2021 23:27:12 - INFO - farm.data_handler.data_silo -       
Preproces

In [7]:
# 4. Create an AdaptiveModel
# a) which consists of a pretrained language model as a basis
language_model = LanguageModel.load(lang_model)
# b) and a prediction head on top that is suited for our task => Text classification
prediction_head = TextClassificationHead(
    class_weights=data_silo.calculate_class_weights(task_name="text_classification"),
    num_labels=len(label_list))

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=0.1,
    lm_output_types=["per_sequence"],
    device=device)

04/22/2021 23:27:27 - INFO - farm.modeling.language_model -   
04/22/2021 23:27:27 - INFO - farm.modeling.language_model -   LOADING MODEL
04/22/2021 23:27:27 - INFO - farm.modeling.language_model -   =============
04/22/2021 23:27:27 - INFO - farm.modeling.language_model -   Could not find bert-base-uncased locally.
04/22/2021 23:27:27 - INFO - farm.modeling.language_model -   Looking on Transformers Model Hub (in local cache and online)...
04/22/2021 23:27:42 - INFO - farm.modeling.language_model -   Loaded bert-base-uncased
04/22/2021 23:27:42 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
04/22/2021 23:27:42 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [1.0476658  0.95648277]


In [9]:
# 5. Create an optimizer
model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    learning_rate=3e-5,
    device=device,
    grad_acc_steps = 2, #2*8 = batch_size 16
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=n_epochs,
    use_amp=use_amp)


04/22/2021 21:06:53 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 3e-05}'
04/22/2021 21:06:53 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
04/22/2021 21:06:53 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 100.0, 'num_training_steps': 1000}'


In [10]:
# 6. Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=n_epochs,
    n_gpu=n_gpu,
    lr_schedule=lr_schedule,
    evaluate_every=evaluate_every,
    device=device,
    grad_acc_steps=2
)


In [11]:
# 7. Let it grow
trainer.train()

04/22/2021 21:06:53 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 245/245 [00:19<00:00, 12.50it/s]
04/22/2021 21:07:18 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 245/245 [00:19<00:00, 12.40it/s]
04/22/2021 21:09:22 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 120 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

04/22/2021 21:09:22 - INFO - farm.eval -   
 _________ text_classification _________
04/22/2021 21:09:22 - INFO - farm.eval -   loss: 0.1259532595750262
04/22/2021 21:09:22 - INFO - farm.eval -   task_name: text_classification
04/22/2021 21:09:22 - INFO - farm.eval -   acc: 0.9591628381827463
04/22/2021 21:09:22 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        REAL     0.9705    0.9436    0.956

04/22/2021 21:11:26 - INFO - farm.eval -   
 _________ text_classification _________
04/22/2021 21:11:26 - INFO - farm.eval -   loss: 0.1052856659103088
04/22/2021 21:11:26 - INFO - farm.eval -   task_name: text_classification
04/22/2021 21:11:26 - INFO - farm.eval -   acc: 0.964267483409903
04/22/2021 21:11:26 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        REAL     0.9749    0.9500    0.9623      1881
       OTHER     0.9549    0.9774    0.9660      2037

    accuracy                         0.9643      3918
   macro avg     0.9649    0.9637    0.9642      3918
weighted avg     0.9645    0.9643    0.9642      3918

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 245/245 [00:19<00:00, 12.34it/s]
04/22/2021 21:11:51 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***********************************

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 245/245 [00:19<00:00, 12.34it/s]
04/22/2021 21:13:55 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 340 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

04/22/2021 21:13:55 - INFO - farm.eval -   
 _________ text_classification _________
04/22/2021 21:13:55 - INFO - farm.eval -   loss: 0.10277282499868448
04/22/2021 21:13:55 - INFO - farm.eval -   task_name: text_classification
04/22/2021 21:13:55 - INFO - farm.eval -   acc: 0.9624808575803981
04/22/2021 21:13:55 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        REAL     0.9309    0.9957    0.96

04/22/2021 21:16:00 - INFO - farm.eval -   
 _________ text_classification _________
04/22/2021 21:16:00 - INFO - farm.eval -   loss: 0.06109759674967096
04/22/2021 21:16:00 - INFO - farm.eval -   task_name: text_classification
04/22/2021 21:16:00 - INFO - farm.eval -   acc: 0.978305257784584
04/22/2021 21:16:00 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        REAL     0.9658    0.9899    0.9777      1881
       OTHER     0.9905    0.9676    0.9789      2037

    accuracy                         0.9783      3918
   macro avg     0.9781    0.9787    0.9783      3918
weighted avg     0.9786    0.9783    0.9783      3918

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 245/245 [00:19<00:00, 12.34it/s]
04/22/2021 21:16:24 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
**********************************

...

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 245/245 [00:19<00:00, 12.55it/s]
04/22/2021 21:40:54 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 1660 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

04/22/2021 21:40:54 - INFO - farm.eval -   
 _________ text_classification _________
04/22/2021 21:40:54 - INFO - farm.eval -   loss: 0.03499812167054665
04/22/2021 21:40:54 - INFO - farm.eval -   task_name: text_classification
04/22/2021 21:40:54 - INFO - farm.eval -   acc: 0.9877488514548239
04/22/2021 21:40:54 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        REAL     0.9878    0.9867    0.9

04/22/2021 21:42:56 - INFO - farm.eval -   
 _________ text_classification _________
04/22/2021 21:42:56 - INFO - farm.eval -   loss: 0.03150899355079182
04/22/2021 21:42:56 - INFO - farm.eval -   task_name: text_classification
04/22/2021 21:42:56 - INFO - farm.eval -   acc: 0.989025012761613
04/22/2021 21:42:56 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        REAL     0.9852    0.9920    0.9886      1881
       OTHER     0.9926    0.9863    0.9894      2037

    accuracy                         0.9890      3918
   macro avg     0.9889    0.9891    0.9890      3918
weighted avg     0.9890    0.9890    0.9890      3918

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 245/245 [00:19<00:00, 12.51it/s]
04/22/2021 21:43:21 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
**********************************

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 245/245 [00:19<00:00, 12.58it/s]
04/22/2021 21:45:23 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 1880 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

04/22/2021 21:45:23 - INFO - farm.eval -   
 _________ text_classification _________
04/22/2021 21:45:23 - INFO - farm.eval -   loss: 0.031287431642874286
04/22/2021 21:45:23 - INFO - farm.eval -   task_name: text_classification
04/22/2021 21:45:23 - INFO - farm.eval -   acc: 0.9895354772843288
04/22/2021 21:45:23 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        REAL     0.9857    0.9926    0.

04/22/2021 21:47:25 - INFO - farm.eval -   
 _________ text_classification _________
04/22/2021 21:47:25 - INFO - farm.eval -   loss: 0.031474922374109565
04/22/2021 21:47:25 - INFO - farm.eval -   task_name: text_classification
04/22/2021 21:47:25 - INFO - farm.eval -   acc: 0.9903011740684022
04/22/2021 21:47:25 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        REAL     0.9878    0.9920    0.9899      1881
       OTHER     0.9926    0.9887    0.9907      2037

    accuracy                         0.9903      3918
   macro avg     0.9902    0.9904    0.9903      3918
weighted avg     0.9903    0.9903    0.9903      3918

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 421/421 [00:33<00:00, 12.55it/s]
04/22/2021 21:48:03 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
********************************

AdaptiveModel(
  (language_model): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
            

In [12]:
# 8. Hooray! You have a model. Store it:
save_dir = Path("./saved_models/bert-base-uncased-fakenews")
model.save(save_dir)
processor.save(save_dir)

In [13]:
# 9. Load it & harvest your fruits (Inference)
basic_texts = [
    {"text": "Biden says he wants to run as president in an interview."},
    {"text": "Biden is sure that Trump is part of a conspiracy to poisen the water in africa."},
]
model = Inferencer.load(save_dir)
result = model.inference_from_dicts(dicts=basic_texts)
print(result)
model.close_multiprocessing_pool()

04/22/2021 21:48:04 - INFO - farm.utils -   Using device: CPU 
04/22/2021 21:48:04 - INFO - farm.utils -   Number of GPUs: 0
04/22/2021 21:48:04 - INFO - farm.utils -   Distributed Training: False
04/22/2021 21:48:04 - INFO - farm.utils -   Automatic Mixed Precision: None
04/22/2021 21:48:04 - INFO - farm.modeling.language_model -   
04/22/2021 21:48:04 - INFO - farm.modeling.language_model -   LOADING MODEL
04/22/2021 21:48:04 - INFO - farm.modeling.language_model -   =============
04/22/2021 21:48:04 - INFO - farm.modeling.language_model -   Model found locally at saved_models\bert-base-uncased-fakenews
04/22/2021 21:48:06 - INFO - farm.modeling.language_model -   Loaded saved_models\bert-base-uncased-fakenews
04/22/2021 21:48:06 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
04/22/2021 21:48:06 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
04/22/2021 21:48:06 - INFO - farm.modeling.prediction_he

[{'task': 'text_classification', 'predictions': [{'start': None, 'end': None, 'context': 'Biden says he wants to run as president in an interview.', 'label': 'REAL', 'probability': 0.99964833}, {'start': None, 'end': None, 'context': 'Biden is sure that Trump is part of a conspiracy to poisen the water in africa.', 'label': 'OTHER', 'probability': 0.9674676}]}]
